<a href="https://colab.research.google.com/github/ArtLapin/Coding/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 1s (7,009 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
import pandas as pd
import numpy as np
import nltk

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
DATA_DIR = "./data"
os.makedirs

<function os.makedirs(name, mode=511, exist_ok=False)>

In [ ]:
import torchtext
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', path=os.path.join(DATA_DIR, 'review.txt'))

14.6MB [00:00, 65.2MB/s]


'/content/data/review.txt'

In [ ]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 29.1 MB/s eta 0:00:00


In [ ]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv(os.path.join(DATA_DIR, 'review.txt'), sep='\t')

# 데이터프레임의 크기
print(data.shape)

# 첫 5행을 출력
data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 긍정 리뷰를 하나 선택
sample_text = data['document'].iloc[149978]
print(type(sample_text))
print(sample_text)

<class 'str'>
그리 만족스럽진못했어도 7점은 나와야되는것같아 10점줌. 주인공들연기도 훌륭했고 내용도 이정도면 괜찮았다. 해피엔딩으로끝났으면 그저 진부한영화가 되버릴뻔. 그래도 엔딩이 섭섭한건 없지않아있었음. 킬러들의도시도그렇고 콜린파렐을 너무 축축하게 끝내버리는듯


In [ ]:
# 트위터 형태소 분석기(Okt)를 활용하여 토큰화화
# 불용어 제거

from konlpy.utils import pprint
from konlpy.tag import Okt
from konlpy.tag import Kkma
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# (단어, 품사) 추출
tokens = okt.pos(sample_text)
pprint(tokens)

[('그리', 'Verb'),
 ('만족스럽진못', 'Adjective'),
 ('했어도', 'Verb'),
 ('7', 'Number'),
 ('점', 'Noun'),
 ('은', 'Josa'),
 ('나와야', 'Verb'),
 ('되는것', 'Verb'),
 ('같아', 'Adjective'),
 ('10', 'Number'),
 ('점줌', 'Noun'),
 ('.', 'Punctuation'),
 ('주인공', 'Noun'),
 ('들', 'Suffix'),
 ('연', 'Modifier'),
 ('기도', 'Noun'),
 ('훌륭했고', 'Adjective'),
 ('내용', 'Noun'),
 ('도', 'Josa'),
 ('이', 'Determiner'),
 ('정도', 'Noun'),
 ('면', 'Josa'),
 ('괜찮았다', 'Adjective'),
 ('.', 'Punctuation'),
 ('해피엔딩', 'Noun'),
 ('으로', 'Josa'),
 ('끝났으면', 'Verb'),
 ('그저', 'Adverb'),
 ('진부한', 'Adjective'),
 ('영화', 'Noun'),
 ('가', 'Josa'),
 ('되', 'Verb'),
 ('버릴', 'Verb'),
 ('뻔', 'Noun'),
 ('.', 'Punctuation'),
 ('그래도', 'Adverb'),
 ('엔딩', 'Noun'),
 ('이', 'Josa'),
 ('섭섭한건', 'Adjective'),
 ('없지', 'Adjective'),
 ('않아있었음', 'Verb'),
 ('.', 'Punctuation'),
 ('킬러', 'Noun'),
 ('들', 'Suffix'),
 ('의', 'Josa'),
 ('도시', 'Noun'),
 ('도', 'Josa'),
 ('그렇고', 'Adjective'),
 ('콜린파렐', 'Noun'),
 ('을', 'Josa'),
 ('너무', 'Adverb'),
 ('축축하게', 'Adjective'),
 ('끝내', 'No

In [ ]:
# 단어 ONLY 추출
tokens = okt.morphs(sample_text)
pprint(tokens)

['그리',
 '만족스럽진못',
 '했어도',
 '7',
 '점',
 '은',
 '나와야',
 '되는것',
 '같아',
 '10',
 '점줌',
 '.',
 '주인공',
 '들',
 '연',
 '기도',
 '훌륭했고',
 '내용',
 '도',
 '이',
 '정도',
 '면',
 '괜찮았다',
 '.',
 '해피엔딩',
 '으로',
 '끝났으면',
 '그저',
 '진부한',
 '영화',
 '가',
 '되',
 '버릴',
 '뻔',
 '.',
 '그래도',
 '엔딩',
 '이',
 '섭섭한건',
 '없지',
 '않아있었음',
 '.',
 '킬러',
 '들',
 '의',
 '도시',
 '도',
 '그렇고',
 '콜린파렐',
 '을',
 '너무',
 '축축하게',
 '끝내',
 '버리는듯']


In [ ]:
# 명사 ONLY 추출
tokens = okt.nouns(sample_text)
pprint(tokens)

['점',
 '점줌',
 '주인공',
 '기도',
 '내용',
 '정도',
 '해피엔딩',
 '영화',
 '뻔',
 '엔딩',
 '킬러',
 '도시',
 '콜린파렐',
 '끝내']


In [ ]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 10000개의 샘플을 선택하여 추출
review_data = review_data[:10000]

# 배열의 크기
print(review_data.shape)

# 첫 번째 데이터
print(review_data[0])

(10000,)
아 더빙.. 진짜 짜증나네요 목소리


In [ ]:
# 세 글자 이상의 명사를 사용
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[:5])

100%|██████████| 10000/10000 [00:32<00:00, 311.67it/s]

10000
['목소리', '포스터', '', '교도소 이야기', '스파이더맨 커스틴 던스트']


In [ ]:
# 사이킷런 패키지 활용
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(10000, 36154)
  (0, 11970)	0.5426222905192297
  (0, 30773)	0.5687563616830532
  (0, 30539)	0.27339898013187175
  (0, 8466)	0.5543737444536609


In [ ]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서10개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 10 })

36154
{'더빙': 8466, '진짜': 30539, '짜증나네요': 30773, '목소리': 11970, '포스터보고': 33619, '초딩영화줄': 31537, '오버연기조차': 23452, '가볍지': 1805, '않구나': 20510, '너무재밓었다그래서보는것을추천한다': 6841}


In [ ]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() }

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{8466: '더빙', 30539: '진짜', 30773: '짜증나네요', 11970: '목소리', 33619: '포스터보고'}


In [ ]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

'목소리 짜증나네요 진짜 더빙'

In [ ]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)

# TF-IDF 벡터를 입력하여 모델 학습
lda.fit(review_tfid)

# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 36154개의 단어)
print(lda.components_.shape)

# 36154개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

(2, 36154)
토픽 유형 1: [('영화', 140.172), ('정말', 64.385), ('진짜', 56.153), ('최고의', 47.434), ('최고', 47.38), ('너무', 45.46), ('그냥', 36.232), ('재밌다', 31.805), ('이런', 30.407), ('이게', 29.684)]
토픽 유형 2: [('영화', 63.742), ('너무', 48.945), ('정말', 43.667), ('진짜', 28.698), ('완전', 25.395), ('별로', 24.889), ('ㅋㅋ', 23.863), ('없고', 23.569), ('쓰레기', 23.344), ('이건', 23.002)]


In [ ]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 36154개의 단어)
print(lda.components_.shape)

(2, 36154)


In [ ]:
# 36154개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('영화', 126.152), ('정말', 80.4), ('진짜', 54.789), ('너무', 54.448), ('최고', 47.25), ('이런', 40.091), ('보고', 32.908), ('이게', 30.934), ('영화가', 29.951), ('이건', 27.986)]
토픽 유형 2: [('영화', 77.762), ('너무', 39.957), ('ㅋㅋ', 34.524), ('이거', 34.389), ('재밌다', 31.77), ('최고의', 31.451), ('진짜', 30.062), ('정말', 27.652), ('그냥', 26.502), ('ㅋㅋㅋ', 23.994)]


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=318317a8ce375af7ca8b5286cb0eeae103bba14540555eace69dd02b059e7747
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=0bb5e3f2abf39f563c016e058552d5d93140528f4b0fe8bbc30c5fe838b48333
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


In [ ]:
labels = data['label'].iloc[:10000].values
print(labels.shape)
print(labels[:10])

(10000,)
[0 1 0 0 1 0 0 0 1 1]


In [ ]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습
lr.fit(review_tfid, labels)

LogisticRegression()

In [ ]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[0]


In [ ]:
test = pd.read_csv('slamdunk.csv')

test.head()

,movie,sentence,score
0,더 퍼스트 슬램덩크,내가본 애니메이션 영화중 &amp;#39;최고&amp;#39;의 영화.,10
1,더 퍼스트 슬램덩크,난 그냥저냥각자 케릭터개성과 농구실력이 다운그레이드된느낌,6
2,더 퍼스트 슬램덩크,마지막부분 빼고는 약간 지루하고 계속 흐름상 왔다갔다해서 집중하기가 힘들었어요그래도...,7
3,더 퍼스트 슬램덩크,진짜 작품 자체로만은 무조건 10점인데 더빙판 성우진이 영...; 본래 성우진들 기...,8
4,더 퍼스트 슬램덩크,원작도 모르고 봤지만 재밌었음 근데 정대만 자꾸 생각남,8


In [ ]:
# 리뷰를 추출
test['sentence']


0                 내가본 애니메이션 영화중 &amp;#39;최고&amp;#39;의 영화.
1                        난 그냥저냥각자 케릭터개성과 농구실력이 다운그레이드된느낌 
2       마지막부분 빼고는 약간 지루하고 계속 흐름상 왔다갔다해서 집중하기가 힘들었어요그래도...
3       진짜 작품 자체로만은 무조건 10점인데 더빙판 성우진이 영...; 본래 성우진들 기...
4                          원작도 모르고 봤지만 재밌었음 근데 정대만 자꾸 생각남
                              ...                        
9293           몇십년만에 보는 슬램덩크 영상미 스토리 뭐 하나 빠지질 않아요. 강추합니다.
9294                                         진짜 추억 그 자체..
9295                              진짜 이건 더빙판으로 한번 더 봐야겠다. 
9296                                   내용을 알고봐도 이정도라니..미침
9297                                       걍 최고...말이 필요없다
Name: sentence, Length: 9298, dtype: object

In [ ]:
def predict(i):
  test_sample = test['sentence'][i]

  # 한글을 제외하고 전부 제거
  test_sample = re.sub(r"[^가-힣]", "", test_sample)

  # 두 글자 이상의 명사만 추출
  tokens = okt.nouns(test_sample)
  cleaned_tokens = []
  for word in tokens:
      if len(word) > 1:
          cleaned_tokens.append(word)
      else:
          pass
  cleaned_review = " ".join(cleaned_tokens)

  # TF-IDF 변환기에 데이터를 입력하여 변환
  test_review_tfid = tfid.transform([cleaned_review])

  # 로지스틱 회귀 모델을 활용하여 분류 예측
  test_pred = lr.predict(test_review_tfid)[0]
  return test_pred


In [ ]:
positive_correct = 0 # 긍정 정답
negative_correct = 0 # 부정 정답
positive_wrong = 0 # 긍정인데 부정으로 답함
negative_wrong = 0 # 부정인데 긍정으로 답함

for i in range(1000):
  print(test['sentence'][i])
  test_pred = predict(i)
  print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))
  print(test['score'][i])
  if test_pred  > 0:
    if test['score'][i] > 6:  # 긍정리뷰를 10 ~ 7 로 가정 (평균이 9.4인걸 감안)
      positive_correct += 1
    else:
      negative_correct += 1
  else:
    if test['score'][i] > 6:  # 긍정리뷰를 10 ~ 7 로 가정
      positive_wrong += 1
    else:
      negative_wrong += 1
  print('----------------------------------------------------------------------')

내가본 애니메이션 영화중 &amp;#39;최고&amp;#39;의 영화.
분석 결과 긍정적인 리뷰로 예측됩니다. 
10
----------------------------------------------------------------------
난 그냥저냥각자 케릭터개성과 농구실력이 다운그레이드된느낌 
분석 결과 부정적인 리뷰로 예측됩니다. 
6
----------------------------------------------------------------------
마지막부분 빼고는 약간 지루하고 계속 흐름상 왔다갔다해서 집중하기가 힘들었어요그래도 마지막부분은 잼있었네요
분석 결과 긍정적인 리뷰로 예측됩니다. 
7
----------------------------------------------------------------------
진짜 작품 자체로만은 무조건 10점인데 더빙판 성우진이 영...; 본래 성우진들 기용하는게 몇억배 좋았을거 같은데 기존 성우진 섭외 실패라도 한건가; 
분석 결과 긍정적인 리뷰로 예측됩니다. 
8
----------------------------------------------------------------------
원작도 모르고 봤지만 재밌었음 근데 정대만 자꾸 생각남
분석 결과 부정적인 리뷰로 예측됩니다. 
8
----------------------------------------------------------------------
명작은 그 결과를 알아도 다시 찾게 만든다
분석 결과 긍정적인 리뷰로 예측됩니다. 
10
----------------------------------------------------------------------
마지막에 울엇다……… 송태섭의 서사에 울고나와서 정대만에게 빠지는 영화
분석 결과 긍정적인 리뷰로 예측됩니다. 
10
----------------------------------------------------------------------
원작본적도 없는데도 불

In [ ]:
print (positive_correct)
print (negative_correct)
print (positive_wrong)
print (negative_wrong)

620
25
308
47
